In [6]:
import numpy as np
from collections import Counter, deque
from scipy.ndimage import label

## Part 1

In [7]:
def read_text_file_as_grid(file_path):
    with open(file_path, 'r') as file:
        grid = [list(line.strip()) for line in file if line.strip()]
    return np.array(grid)

In [8]:
input_file = "example1.txt"
grid = read_text_file_as_grid(input_file)

In [9]:
def find_regions_and_edges(grid):
    rows, cols = grid.shape
    visited = np.zeros_like(grid, dtype=bool)
    results = {}
    region_id = 1
    
    def calculate_cell_edges(x, y, char):
        edges = 0
        for nx, ny in [(x-1, y), (x+1, y), (x, y-1), (x, y+1)]:
            if 0 <= nx < rows and 0 <= ny < cols:
                if grid[nx, ny] != char: 
                    edges += 1
            else:
                edges += 1
        return edges
    
    def bfs(x, y, char):
        queue = deque([(x, y)])
        visited[x, y] = True
        size = 0
        total_edges = 0
        
        while queue:
            cx, cy = queue.popleft()
            size += 1
            total_edges += calculate_cell_edges(cx, cy, char)
            
            for nx, ny in [(cx-1, cy), (cx+1, cy), (cx, cy-1), (cx, cy+1)]:
                if 0 <= nx < rows and 0 <= ny < cols:  # In bounds
                    if not visited[nx, ny] and grid[nx, ny] == char:
                        visited[nx, ny] = True
                        queue.append((nx, ny))
        
        return size, total_edges
    
    for i in range(rows):
        for j in range(cols):
            if not visited[i, j]:
                char = grid[i, j]
                size, edges = bfs(i, j, char)
                results[f'{char}-{region_id}'] = {'elements': size, 'edges': edges}
                region_id += 1
    
    return results

# results = find_regions_and_edges(grid)
# for region, stats in results.items():
#     print(f"Region {region}: {stats}")

In [10]:
input_file = "input.txt"
grid = read_text_file_as_grid(input_file)

results = find_regions_and_edges(grid)

total_cost = 0
for stats in results.values():
    price = stats["elements"] * stats["edges"]
    total_cost += price
    
print(total_cost)

1421958


## Part 2

In [72]:
def find_region_edges(x, y, grid, visited):
    char = grid[x, y]
    region = np.zeros_like(grid, dtype=bool)
    queue = deque([(x, y)])
    visited[x, y] = True 
    size = 0 
    
    while queue:
        cx, cy = queue.popleft()
        region[cx, cy] = True
        size += 1
        
        for nx, ny in [(cx-1, cy), (cx+1, cy), (cx, cy-1), (cx, cy+1)]:
            if 0 <= nx < grid.shape[0] and 0 <= ny < grid.shape[1]:
                if not visited[nx, ny] and grid[nx, ny] == char:
                    visited[nx, ny] = True
                    queue.append((nx, ny))
    
    return region, size


In [64]:
def count_corners(region, grid):
    corners = 0
    rows, cols = grid.shape
    
    for x, y in zip(*np.where(region)):
        char = grid[x, y]
        
        neighbors = [
            (x-1, y), (x+1, y), (x, y-1), (x, y+1),
            (x-1, y-1), (x-1, y+1), (x+1, y-1), (x+1, y+1)
        ]
        
        print(f"char {char} at {x,y}")
        
        boundary_changes = 0
        for nx, ny in neighbors:
            if 0 <= nx < rows and 0 <= ny < cols:
                if grid[nx, ny] != char:
                    # print(f"boundary change at: {nx, ny}, {grid[nx, ny]}")
                    boundary_changes += 1
        
        if boundary_changes >= 2:
            corners += 1

    return corners

In [78]:
input_file = "example2.txt"
grid = read_text_file_as_grid(input_file)
visited = np.zeros_like(grid, dtype=bool)

region_info = []

for x in range(grid.shape[0]):
    for y in range(grid.shape[1]):
        if not visited[x, y]:
            region, size = find_region_edges(x, y, grid, visited)
            corners = count_corners(region, grid)
            region_info.append((size, corners))

# total_cost = 0
# for size, corner in region_info:
#     price = size * corner
#     total_cost += price
    
print(region_info)

[(17, 2), (4, 2), (4, 2)]
